In [ ]:
from transformers import AutoTokenizer, PhiForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import pandas as pd
import torch
from torch.utils.data import Dataset
import os
from deepspeed.runtime.zero.stage_1_and_2 import estimate_zero2_model_states_mem_needs_all_live

#TRAINING

##PREPROCESSING

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("phi-1_5", local_files_only = True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def collect_samples(genre):
    list_ = []
    for filename in os.listdir(f"books/{genre}"):
        with open(f"books/{genre}/{filename}", "r") as file:
            file_content = file.read()
        tokenized = tokenizer(file_content)

        for i in range(0, len(tokenized["input_ids"]), 2048):
            input_ids = tokenized["input_ids"][i:(i + 2048)]
            attention_mask = tokenized["attention_mask"][i:(i + 2048)]
            list_.append({'input_ids': input_ids, 'attention_mask': attention_mask})
    return list_

df_genreA = collect_samples("fantasy")
df_genreB = collect_samples("romance")
df_genreC = collect_samples("sci-fi")
df_genreD = collect_samples("thriller")

In [ ]:
def convert_list_to_dataframe(list_):
    df = pd.DataFrame(list_, columns = ["text"])

    for i in range(len(list_)):
        df.loc[i, "text"] = ""
        df.at[i, 'text'] = list_[i]
    return df

df_genreA = convert_list_to_dataframe(df_genreA)
df_genreB = convert_list_to_dataframe(df_genreB)
df_genreC = convert_list_to_dataframe(df_genreC)
df_genreD = convert_list_to_dataframe(df_genreD)

In [ ]:
df_genreA = df_genreA.sample(n = 2500)
df_genreA_train = df_genreA.sample(n = (int(0.8 * len(df_genreA))))
df_genreA_eval = df_genreA[~df_genreA.index.isin(df_genreA_train.index)]

df_genreB = df_genreB.sample(n = 2500)
df_genreB_train = df_genreB.sample(n = (int(0.8 * len(df_genreB))))
df_genreB_eval = df_genreB[~df_genreB.index.isin(df_genreB_train.index)]

df_genreC = df_genreC.sample(n = 2500)
df_genreC_train = df_genreC.sample(n = (int(0.8 * len(df_genreC))))
df_genreC_eval = df_genreC[~df_genreC.index.isin(df_genreC_train.index)]

df_genreD = df_genreD.sample(n = 2500)
df_genreD_train = df_genreD.sample(n = (int(0.8 * len(df_genreD))))
df_genreD_eval = df_genreD[~df_genreD.index.isin(df_genreD_train.index)]

In [ ]:
df_train = pd.concat([df_genreA_train, df_genreB_train, df_genreC_train, df_genreD_train])
df_train = df_train.sample(frac = 1).reset_index(drop = True)

df_eval = pd.concat([df_genreA_eval, df_genreB_eval, df_genreC_eval, df_genreD_eval])
df_eval = df_eval.sample(frac = 1).reset_index(drop = True)

torch.save(df_train, 'df_train.pt')
torch.save(df_eval, 'df_eval.pt')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("phi-1_5", local_files_only = True)
tokenizer.pad_token = tokenizer.eos_token

df_train = torch.load('df_train.pt')
df_eval = torch.load('df_eval.pt')

class Books3(Dataset):
    def __init__(self, df):
        self.text = []
        for i in range(len(df)):
            self.text.append(df["text"][i])

    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        return self.text[item]

dataset_train = Books3(df_train)
dataset_eval = Books3(df_eval)
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, return_tensors = "pt", mlm = False)

##FINE-TUNING

In [ ]:
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "9994"
os.environ["RANK"] = "0"
os.environ["LOCAL_RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"

ds_config = {
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },
    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto"
        }
    },
    "zero_optimization": {
        "stage": 2,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": True
        },
        "allgather_partitions": True,
        "allgather_bucket_size": 2e8,
        "overlap_comm": True,
        "reduce_scatter": True,
        "reduce_bucket_size": 2e8,
        "contiguous_gradients": True
    },
    "train_micro_batch_size_per_gpu": "auto",
    "train_batch_size": "auto",
    "gradient_accumulation_steps": "auto"
}

In [ ]:
model = PhiForCausalLM.from_pretrained("phi-1_5", local_files_only = True)

model.gradient_checkpointing_enable()
model.config.use_cache = False

estimate_zero2_model_states_mem_needs_all_live(model, num_gpus_per_node = 1, num_nodes = 1)

In [ ]:
args = TrainingArguments(output_dir = "model",
                         overwrite_output_dir = True,
                         per_device_train_batch_size = 4,
                         per_device_eval_batch_size = 1,
                         gradient_accumulation_steps = 1,
                         num_train_epochs = 1,
                         optim = "adamw_torch",
                         learning_rate= 0.00001,
                         warmup_ratio = 0.1,
                         lr_scheduler_type = "linear",
                         fp16 = True,
                         deepspeed = ds_config,
                         save_strategy = "no",
                         logging_strategy = "epoch",
                         evaluation_strategy = "epoch")

trainer = Trainer(model = model,
                  tokenizer = tokenizer,
                  args = args,
                  train_dataset = dataset_train,
                  eval_dataset = dataset_eval,
                  data_collator = data_collator)

In [ ]:
trainer.train()
trainer.save_model()

#INFERENCE

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("model", local_files_only = True)
tokenizer.pad_token = tokenizer.eos_token

model = PhiForCausalLM.from_pretrained("model", local_files_only = True)
model = model.to(torch.device("cuda"))

In [19]:
inputs = tokenizer.encode("In 2067, humanity faces extinction due to a global blight. Joseph Cooper, a former NASA test pilot, along with his son and daughter, Tom and Murph, and father-in-law Donald, toil as subsistence farmers. One evening during a dust storm,", return_tensors = "pt")
inputs = inputs.to(torch.device("cuda"))

outputs = model.generate(inputs, max_new_tokens = 50, do_sample = False)
print(tokenizer.decode(outputs[0]))

In 2067, humanity faces extinction due to a global blight. Joseph Cooper, a former NASA test pilot, along with his son and daughter, Tom and Murph, and father-in-law Donald, toil as subsistence farmers. One evening during a dust storm, they are forced to flee their farm and seek refuge in a nearby town.

The town is a small, sleepy place, with a population of only a few thousand. The town is surrounded by a vast desert, and the only source of water
